In [118]:
import numpy as np
import pandas as pd

In [119]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [120]:
!cp -r drive/MyDrive/newz-aware/ /content/newz-aware

cp: cannot stat 'drive/MyDrive/newz-aware/': No such file or directory


In [121]:
%cd newz-aware/

[Errno 2] No such file or directory: 'newz-aware/'
/content/newz-aware


In [122]:
!pip install -r requirements.txt

In [123]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [124]:
!python -m spacy download en_core_web_sm

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd
     |████████████████████████████████| 13.6 MB 7.6 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [125]:
df = pd.read_csv('TrainingData/scrapedDataset.csv')

X = df['title']
y = df['bias']

In [126]:
import spacy
from nltk.corpus import stopwords
import re

class PreProcessor:

    def __init__(self, lang = 'english'):
        self.nlp = spacy.load('en_core_web_sm')
        self.stop_words = set(stopwords.words(lang))

    def remove_special_chars(self, doc):

        doc = " ".join(re.findall(r'[a-zA-Z0-9]+', doc))
        return doc.lower()

    def lemmatize(self, doc):
        doc = self.nlp(doc)
        tokenized = [token.lemma_ for token in doc]
        return tokenized

    def remove_stop_words(self, tokenized_doc):

        tokenized_doc_no_stopwords = []

        for word in tokenized_doc:
            if word not in self.stop_words:
                tokenized_doc_no_stopwords.append(word)

        return tokenized_doc_no_stopwords

    def forward(self, doc):
        doc = self.remove_special_chars(doc)
        doc = self.lemmatize(doc)
        doc = self.remove_stop_words(doc)

        return " ".join(doc)

preproc = PreProcessor()

for i,x in enumerate(X):
    X[i] = preproc.forward(str(X[i]))
    print(i+1)

X.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Streaming output truncated to the last 5000 lines.
16005
16006
16007
16008
16009
16010
16011
16012
16013
16014
16015
16016
16017
16018
16019
16020
16021
16022
16023
16024
16025
16026
16027
16028
16029
16030
16031
16032
16033
16034
16035
16036
16037
16038
16039
16040
16041
16042
16043
16044
16045
16046
16047
16048
16049
16050
16051
16052
16053
16054
16055
16056
16057
16058
16059
16060
16061
16062
16063
16064
16065
16066
16067
16068
16069
16070
16071
16072
16073
16074
16075
16076
16077
16078
16079
16080
16081
16082
16083
16084
16085
16086
16087
16088
16089
16090
16091
16092
16093
16094
16095
16096
16097
16098
16099
16100
16101
16102
16103
16104
16105
16106
16107
16108
16109
16110
16111
16112
16113
16114
16115
16116
16117
16118
16119
16120
16121
16122
16123
16124
16125
16126
16127
16128
16129
16130
16131
16132
16133
16134
16135
16136
16137
16138
16139
16140
16141
16142
16143
16144
16145
16146
16147
16148
16149
16150
16151
16152
16153
16154
16155
16156
16157
16158
16159
16160
16161
16162
1

0       zimmerman verdict renew focus stand ground law
1    senate immigration bill pass judiciary committ...
2    let doma fool supreme court restrict right was...
3    obamacare trouble exchange provision delay law...
4    exclusive immigration agent rip house lawmaker...
Name: title, dtype: object

In [127]:
y.replace(to_replace = list(range(-4,5)), value=[0,0,0,1,1,1,2,2,2], inplace=True)
y.value_counts()

1    17510
2     1878
0     1616
Name: bias, dtype: int64

In [128]:
df = pd.concat([X,y], axis=1)
df

,title,bias
0,zimmerman verdict renew focus stand ground law,1
1,senate immigration bill pass judiciary committ...,1
2,let doma fool supreme court restrict right was...,1
3,obamacare trouble exchange provision delay law...,1
4,exclusive immigration agent rip house lawmaker...,1
...,...,...
20999,dana milbank senator turn table caymans invest...,2
21000,republicans hatred obama blind public disinter...,2
21001,dana milbank stockman step forward republican ...,1
21002,e j dionne republican problem solver washingto...,2


In [129]:
df.drop(df[df['title']=='nan'].index, inplace=True)

In [130]:
y.value_counts()

1    17510
2     1878
0     1616
Name: bias, dtype: int64

In [131]:
df_class0 = df[df['bias'] == 0]
df_class1 = df[df['bias'] == 1]
df_class2 = df[df['bias'] == 2]

In [132]:
df_class0_oversampled = df_class0.sample(df_class1.shape[0], replace = True)
df_class2_oversampled = df_class2.sample(df_class1.shape[0], replace = True)

print(df_class0_oversampled.shape)
print(df_class2_oversampled.shape)

(12098, 2)
(12098, 2)


In [133]:
df_oversampled = pd.concat([df_class1, df_class0_oversampled, df_class2_oversampled], axis = 0)
print(df_oversampled.shape)
print(df_oversampled['bias'].value_counts())

(36294, 2)
1    12098
0    12098
2    12098
Name: bias, dtype: int64


## Checkpoint Preprocessing done

In [134]:
df = df_oversampled

In [135]:
X = df['title']
y = df['bias']

In [136]:
X

0           zimmerman verdict renew focus stand ground law
1        senate immigration bill pass judiciary committ...
2        let doma fool supreme court restrict right was...
3        obamacare trouble exchange provision delay law...
4        exclusive immigration agent rip house lawmaker...
                               ...                        
18520     hillary clinton silence heckler press civility c
13656                                         minimum wage
20769    must see jon stewart epic takedown ted cruz no...
642      edward snowden come forward source nsa leak wa...
3061                       abortion walk bar say bartender
Name: title, Length: 36294, dtype: object

In [137]:
from sklearn.model_selection import train_test_split

train_X, t_X, train_y, t_y = train_test_split(X, y, test_size=0.2, random_state=101)
test_X, dev_X, test_y, dev_y = train_test_split(t_X, t_y, test_size=0.5, random_state=101)

In [138]:
train_X.shape

(29035,)

In [139]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features = 8000, lowercase=False, ngram_range=(1,2))
train_X_vec = vectorizer.fit_transform(train_X)
dev_X_vec = vectorizer.transform(dev_X)
test_X_vec = vectorizer.transform(test_X)

In [140]:
import joblib

joblib.dump(vectorizer, 'tfidf_for_bias.pkl')

['tfidf_for_bias.pkl']

In [141]:
train_X = train_X_vec.toarray()
test_X = test_X_vec.toarray()
dev_X = dev_X_vec.toarray()

In [142]:
train_X.shape

(29035, 8000)

In [143]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(train_X, train_y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [144]:
import joblib

joblib.dump(classifier, 'naiveBayesModel.pkl')

['naiveBayesModel.pkl']

## Evaluation

In [145]:
dev_pred = classifier.predict(dev_X)

In [146]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(dev_y, dev_pred))
print(classification_report(dev_y, dev_pred))

[[929 298  49]
 [158 940 124]
 [ 65 264 803]]
              precision    recall  f1-score   support

           0       0.81      0.73      0.77      1276
           1       0.63      0.77      0.69      1222
           2       0.82      0.71      0.76      1132

    accuracy                           0.74      3630
   macro avg       0.75      0.74      0.74      3630
weighted avg       0.75      0.74      0.74      3630



In [147]:
test_pred = classifier.predict(test_X)

In [148]:
print(confusion_matrix(test_y, test_pred))
print(classification_report(test_y, test_pred))

[[906 259  47]
 [127 957 137]
 [ 78 283 835]]
              precision    recall  f1-score   support

           0       0.82      0.75      0.78      1212
           1       0.64      0.78      0.70      1221
           2       0.82      0.70      0.75      1196

    accuracy                           0.74      3629
   macro avg       0.76      0.74      0.75      3629
weighted avg       0.76      0.74      0.75      3629

